# 实验一
自定义VGG16网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np

### 定义CNN网络结构



In [12]:
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
 #定义VGG16结构   
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))     # ZeroPadding2D为零填充
model.add(Convolution2D(64, 3, 3, activation='relu'))       # 卷积核个数为64个，卷积核大小为3*3，卷积步长为3，激活函数为relu
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))       # 激活函数为relu
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))       # 激活函数为softmax

#在compile函数里定义loss函数、优化器和metrics
model.compile(loss='binary_crossentropy',               # 指定的损失函数是binary_crossentropy
              optimizer='rmsprop',                      # 采用的优化器是rmsprop         选SGD时，为梯度下降算法
              metrics=['accuracy'])                     # 使用的衡量标准是accuracy

h:\ProgramData\Anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
h:\ProgramData\Anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
h:\ProgramData\Anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
h:\ProgramData\Anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_4/MaxPool' (op: 'MaxPool') with input shapes: [?,1,112,128].

### 查看model架构



In [13]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

### 定义ImageDataGenerator


In [45]:
train_data_dir = r'G:\BaiduNetdiskDownload\百度云文件下载\python培训课课件等\11.06 机器视觉1\作业\dogs-vs-cats\train'
validation_data_dir = r'G:\BaiduNetdiskDownload\百度云文件下载\python培训课课件等\11.06 机器视觉1\作业\dogs-vs-cats\validation'
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 20

#定义ImageDataGenerator为生成training和validation samples

''' 
ImageDataGenerator
    是keras.preprocessing.image模块中的图片生成器，同时也可以在batch中对数据进行增强，扩充数据集大小，增强模型的泛化能力。比如进行旋转，变形，归一化等等。

shear_range(): float, 透视变换的范围        图2
zoom_range(): 缩放范围                      图3
horizontal_flip(): 水平反转                 图4
        所以会得到三张图，一共就有四张图（图1为原图）

'''

train_datgen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [14]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

### 使用训练后模型预测图像





In [48]:
import cv2
img = cv2.resize(cv2.imread(r'G:\BaiduNetdiskDownload\百度云文件下载\python培训课课件等\11.06 机器视觉1\作业\dogs-vs-cats\test\7.jpg'), (img_width, img_height)).astype(np.float32)

x = img_to_array(img)

x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

score = model.predict(x)


print(score)

[[0.99895906]]
